In [ ]:
# <-------------------- CELL 1: IMPORTS -------------------->
print("Cell 1: Imports - Executing...")
import mlflow
import mlflow.pyfunc
import mlflow.sklearn
import mlflow.lightgbm
import mlflow.xgboost
import mlflow.catboost

import pandas as pd
import numpy as np
import os
import time
import joblib 
from typing import List, Dict, Any, Tuple, Union, cast
import itertools # For ensemble combinations

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, log_loss, brier_score_loss, f1_score, precision_score, recall_score, accuracy_score
from sklearn.inspection import PartialDependenceDisplay

import xgboost as xgb
import lightgbm as lgb
import catboost as cb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from scipy.optimize import minimize 

from pyspark.sql import SparkSession

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', message="Previously subsetted data...")
warnings.filterwarnings('ignore', message="Using `tqdm.autonotebook.tqdm`") 
warnings.filterwarnings('ignore', message="The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator") # From UMAP in SHAP if used

if 'spark' not in locals():
    spark = SparkSession.builder.appName("Classification_Training_Ensemble_MVP").getOrCreate()
    print("SparkSession created.")
else:
    print("SparkSession already exists.")

print("Imports successful for Training & Ensembling Pipeline.")
print("-" * 50)

# <-------------------- CELL 2: INIT CELL - GLOBAL CONFIGURATIONS FOR TRAINING -------------------->
print("\nCell 2: Global Configurations for Training - Executing...")

# --- MLflow Configuration ---
# !!! IMPORTANT: SET YOUR MLFLOW EXPERIMENT PATH !!!
MLFLOW_EXPERIMENT_PATH = "/Users/your_username@example.com/MVP_Conversion_PriceSensitivity_Training_Full" # CHANGE THIS

# --- Data Paths (Unity Catalog Volumes - Processed Data from Preprocessing Script) ---
# These paths MUST point to the Parquet files with named, transformed columns
# created by your Pandas preprocessing script.
# !!! IMPORTANT: VERIFY THESE PATHS EXACTLY MATCH THE OUTPUT OF YOUR PREPROCESSING SCRIPT !!!
UC_BASE_DATA_PATH_TRAINING = "/Volumes/delfos/" # From your spec, ensure this is the base
PROCESSED_DATA_VERSION_TRAINING = "v1_pandas_fe_final_output" # Ensure matches preproc output version
PROCESSED_DATA_SUBDIR_TRAINING = "processed_data" # Ensure matches preproc output subdir

PROCESSED_DATA_DIR_VERSIONED_TRAINING = os.path.join(UC_BASE_DATA_PATH_TRAINING, PROCESSED_DATA_SUBDIR_TRAINING, PROCESSED_DATA_VERSION_TRAINING)

SHARED_PROCESSED_TRAIN_PATH = os.path.join(PROCESSED_DATA_DIR_VERSIONED_TRAINING, "train_processed_named_cols.parquet")
SHARED_PROCESSED_TEST_PATH = os.path.join(PROCESSED_DATA_DIR_VERSIONED_TRAINING, "test_processed_named_cols.parquet")

# --- Target and Key Feature Names ---
# !!! IMPORTANT: SET YOUR ACTUAL TARGET & PREMIUM COLUMN NAMES (as they appear in the processed Parquet) !!!
TARGET_COLUMN_NAME = "target_binary" # This is your y
PREMIUM_COLUMN_NAME = "premium_col" # This is one of your X features (processed)

# --- Output Paths for OOF/Test Predictions (Parquet) from Base Models ---
OOF_PREDS_SUBDIR_TRAINING = "oof_predictions_classification_v1_mvp"
TEST_PREDS_SUBDIR_TRAINING = "test_predictions_classification_v1_mvp"
OOF_PREDS_DIR_UCV = os.path.join(UC_BASE_DATA_PATH_TRAINING, OOF_PREDS_SUBDIR_TRAINING)
TEST_PREDS_DIR_UCV = os.path.join(UC_BASE_DATA_PATH_TRAINING, TEST_PREDS_SUBDIR_TRAINING)

# --- HPO Configuration ---
NUM_HPO_TRIALS_PER_ALGO = 10 # !!! START LOW (e.g., 3-5) FOR TESTING, INCREASE LATER (e.g., 25-50) !!!
HPO_OPTIMIZATION_METRIC = "auc_roc" # Options: 'auc_roc', 'auc_pr', 'log_loss', 'brier_score'. Loss will be -metric if maximizing.

# --- Base Algorithms to Run ---
BASE_ALGORITHMS_TO_TRAIN = ['lightgbm', 'xgboost', 'catboost'] # As requested

# --- Cross-Validation for OOF ---
K_FOLDS_OOF = 5

# --- Reproducibility ---
GLOBAL_SEED = 117

# --- Ensemble Configuration ---
# Will generate combinations from BASE_ALGORITHMS_TO_TRAIN

# --- Other Global Settings ---
MAX_METRICS_TO_LOG_PER_RUN = 7
PDP_N_GRID_POINTS = 20
PDP_PERCENTILES = (0.05, 0.95) # For PDP grid range

# Ensure output directories for OOF/Test predictions exist
try:
    os.makedirs(OOF_PREDS_DIR_UCV, exist_ok=True)
    os.makedirs(TEST_PREDS_DIR_UCV, exist_ok=True)
    print(f"Checked/created OOF directory: {OOF_PREDS_DIR_UCV}")
    print(f"Checked/created Test Preds directory: {TEST_PREDS_DIR_UCV}")
except Exception as e:
    print(f"Warning: Could not create OOF/Test Preds directories. Error: {e}")

print(f"--- Training Global Configurations Initialized ---")
# ... (Print key configs)
print(f"MLflow Experiment Path: {MLFLOW_EXPERIMENT_PATH}")
print(f"Processed Train Data Path: {SHARED_PROCESSED_TRAIN_PATH}")
print(f"Processed Test Data Path: {SHARED_PROCESSED_TEST_PATH}")
print(f"Base Algorithms: {BASE_ALGORITHMS_TO_TRAIN}")
print(f"HPO Trials per Algo: {NUM_HPO_TRIALS_PER_ALGO}")
print(f"HPO Optimization Metric: {HPO_OPTIMIZATION_METRIC.upper()}")
print("-" * 50)


# <-------------------- CELL 3: UTILITY FUNCTIONS & CORE LOGIC (HPO, OOF, ENSEMBLE) -------------------->
print("\nCell 3: Utility Functions & Core Logic - Defining...")

# --- MLflow Utility ---
def get_or_create_experiment(experiment_name_param, spark_session_param=None):
    try:
        experiment = mlflow.get_experiment_by_name(experiment_name_param)
        if experiment: print(f"MLflow experiment '{experiment_name_param}' found with ID: {experiment.experiment_id}"); return experiment.experiment_id
        else:
            print(f"MLflow experiment '{experiment_name_param}' not found. Creating."); experiment_id = mlflow.create_experiment(name=experiment_name_param)
            print(f"MLflow experiment '{experiment_name_param}' created with ID: {experiment_id}"); return experiment_id
    except Exception as e: print(f"Error in get_or_create_experiment for '{experiment_name_param}': {e}"); return None

# --- Algorithm Search Spaces (Classifiers) ---
ALGORITHM_CLASSIFIER_SEARCH_SPACES = {
    'lightgbm': { 'model_params': {
            'n_estimators': hp.quniform('lgbm_n_estimators', 50, 250, 25), 'learning_rate': hp.loguniform('lgbm_learning_rate', np.log(0.01), np.log(0.1)),
            'num_leaves': hp.quniform('lgbm_num_leaves', 20, 100, 5), 'max_depth': hp.quniform('lgbm_max_depth', 3, 10, 1),
            'subsample': hp.uniform('lgbm_subsample', 0.6, 1.0), 'colsample_bytree': hp.uniform('lgbm_colsample_bytree', 0.6, 1.0),
            'reg_alpha': hp.uniform('lgbm_reg_alpha', 0.0, 0.5), 'reg_lambda': hp.uniform('lgbm_reg_lambda', 0.0, 0.5),
            'class_weight': hp.choice('lgbm_class_weight', [None, 'balanced'])
    }},
    'xgboost': { 'model_params': {
            'n_estimators': hp.quniform('xgb_n_estimators', 50, 250, 25), 'learning_rate': hp.loguniform('xgb_learning_rate', np.log(0.01), np.log(0.1)),
            'max_depth': hp.quniform('xgb_max_depth', 3, 10, 1), 'subsample': hp.uniform('xgb_subsample', 0.6, 1.0),
            'colsample_bytree': hp.uniform('xgb_colsample_bytree', 0.6, 1.0), 'gamma': hp.uniform('xgb_gamma', 0, 0.3),
            'reg_alpha': hp.uniform('xgb_reg_alpha', 0.0, 0.5),'reg_lambda': hp.uniform('xgb_reg_lambda', 0.0, 0.5),
            'scale_pos_weight': hp.quniform('xgb_scale_pos_weight', 1, 10, 1) # For imbalanced classes
    }},
    'catboost': { 'model_params': {
            'iterations': hp.quniform('cb_iterations', 50, 250, 25), 'learning_rate': hp.loguniform('cb_learning_rate', np.log(0.01), np.log(0.1)),
            'depth': hp.quniform('cb_depth', 3, 10, 1), 'l2_leaf_reg': hp.loguniform('cb_l2_leaf_reg', np.log(1), np.log(9)),
            'border_count': hp.quniform('cb_border_count', 32, 254, 32), # Max 254 for CPU
            'subsample': hp.uniform('cb_subsample', 0.6, 1.0) # If bootstrap_type supports it
    }}
}
print("Classifier Search spaces defined.")

# --- Helper to Load Processed Parquet and Prepare Sklearn X, y ---
def load_processed_parquet_to_sklearn(parquet_path: str, label_col: str) -> Tuple[pd.DataFrame, pd.Series, List[str]]:
    print(f"    Loading Parquet for sklearn: {parquet_path}")
    pdf = pd.read_parquet(parquet_path)
    if label_col not in pdf.columns:
        print(f"    Label column '{label_col}' not found. Assuming all columns in Parquet are features.")
        X_pdf = pdf; y_series = None
    else:
        y_series = pdf[label_col].astype(int); X_pdf = pdf.drop(columns=[label_col])
    feature_names_ordered = X_pdf.columns.tolist()
    print(f"      Loaded X_pdf shape: {X_pdf.shape}, y_series shape: {y_series.shape if y_series is not None else 'N/A'}, Features: {len(feature_names_ordered)}")
    return X_pdf, y_series, feature_names_ordered

# --- HPO Objective Function (Classification) ---
global HPO_PARENT_RUN_ID_HPO_OBJ, CURRENT_ALGORITHM_TYPE_HPO_OBJ, FEATURE_NAMES_FOR_HPO_OBJ
HPO_PARENT_RUN_ID_HPO_OBJ = None; CURRENT_ALGORITHM_TYPE_HPO_OBJ = None; FEATURE_NAMES_FOR_HPO_OBJ = []

def objective_function_classification(hyperparams_from_hyperopt):
    global HPO_PARENT_RUN_ID_HPO_OBJ, CURRENT_ALGORITHM_TYPE_HPO_OBJ, FEATURE_NAMES_FOR_HPO_OBJ
    global SHARED_PROCESSED_TRAIN_PATH, SHARED_PROCESSED_TEST_PATH, TARGET_COLUMN_NAME, PREMIUM_COLUMN_NAME
    global HPO_OPTIMIZATION_METRIC, GLOBAL_SEED, MAX_METRICS_TO_LOG_PER_RUN

    sanitized_hyperparams = {k: v.item() if isinstance(v, np.generic) else int(v) if k in ['max_depth', 'depth', 'n_estimators', 'num_leaves', 'iterations', 'border_count', 'scale_pos_weight'] and v is not None else (int(v) if isinstance(v,float) and v.is_integer() else v) for k,v in hyperparams_from_hyperopt.items()}
    if 'max_depth' in sanitized_hyperparams and sanitized_hyperparams['max_depth'] is None: pass # Allow None for max_depth
    elif 'max_depth' in sanitized_hyperparams: sanitized_hyperparams['max_depth'] = int(sanitized_hyperparams['max_depth'])
    if 'depth' in sanitized_hyperparams: sanitized_hyperparams['depth'] = int(sanitized_hyperparams['depth'])


    trial_run_name = f"Trial_{CURRENT_ALGORITHM_TYPE_HPO_OBJ}_{time.strftime('%Y%m%d-%H%M%S')}_{os.urandom(4).hex()}"
    with mlflow.start_run(run_name=trial_run_name, nested=True) as trial_run:
        if HPO_PARENT_RUN_ID_HPO_OBJ: mlflow.set_tag("parent_hpo_campaign_run_id", HPO_PARENT_RUN_ID_HPO_OBJ)
        mlflow.log_param("model_type_trial", CURRENT_ALGORITHM_TYPE_HPO_OBJ); mlflow.log_params(sanitized_hyperparams)
        mlflow.set_tag("seed", GLOBAL_SEED); mlflow.log_param("train_data_hpo", SHARED_PROCESSED_TRAIN_PATH); mlflow.log_param("test_data_hpo", SHARED_PROCESSED_TEST_PATH)

        try:
            X_train_pdf, y_train_series, train_feature_names = load_processed_parquet_to_sklearn(SHARED_PROCESSED_TRAIN_PATH, TARGET_COLUMN_NAME)
            X_test_pdf, y_test_series, _ = load_processed_parquet_to_sklearn(SHARED_PROCESSED_TEST_PATH, TARGET_COLUMN_NAME) # Test feature names should match train
            if y_test_series is None: raise ValueError("Test labels required for HPO evaluation.")
            
            X_train_np, y_train_np = X_train_pdf.values, y_train_series.values
            X_test_np, y_test_np = X_test_pdf.values, y_test_series.values
            FEATURE_NAMES_FOR_HPO_OBJ = list(train_feature_names) # Capture for this trial scope for PDP

            monotone_constraints_val = None
            if PREMIUM_COLUMN_NAME in FEATURE_NAMES_FOR_HPO_OBJ:
                premium_idx = FEATURE_NAMES_FOR_HPO_OBJ.index(PREMIUM_COLUMN_NAME)
                monotone_constraints_val = [0] * len(FEATURE_NAMES_FOR_HPO_OBJ)
                monotone_constraints_val[premium_idx] = -1
                mlflow.log_param("monotonicity_applied", f"{PREMIUM_COLUMN_NAME}_idx{premium_idx}_is_-1")
            
            model_params_for_fit = sanitized_hyperparams.copy()
            model = None
            if CURRENT_ALGORITHM_TYPE_HPO_OBJ == 'lightgbm':
                if monotone_constraints_val: model_params_for_fit['monotone_constraints'] = monotone_constraints_val
                model = lgb.LGBMClassifier(**model_params_for_fit, random_state=GLOBAL_SEED, n_jobs=-1, verbose=-1)
            elif CURRENT_ALGORITHM_TYPE_HPO_OBJ == 'xgboost':
                if monotone_constraints_val: model_params_for_fit['monotone_constraints'] = tuple(monotone_constraints_val)
                model = xgb.XGBClassifier(**model_params_for_fit, random_state=GLOBAL_SEED, use_label_encoder=False, eval_metric='logloss', n_jobs=-1)
            elif CURRENT_ALGORITHM_TYPE_HPO_OBJ == 'catboost':
                if monotone_constraints_val: model_params_for_fit['monotone_constraints'] = monotone_constraints_val
                model = cb.CatBoostClassifier(**model_params_for_fit, random_state=GLOBAL_SEED, verbose=0, allow_writing_files=False)
            else: raise ValueError(f"Unsupported model type: {CURRENT_ALGORITHM_TYPE_HPO_OBJ}")
            
            model.fit(X_train_np, y_train_np)
            pred_proba = model.predict_proba(X_test_np)[:, 1]
            pred_labels = (pred_proba >= 0.5).astype(int)

            auc_roc = roc_auc_score(y_test_np, pred_proba); auc_pr_val = average_precision_score(y_test_np, pred_proba)
            logloss = log_loss(y_test_np, pred_proba); brier = brier_score_loss(y_test_np, pred_proba)
            f1 = f1_score(y_test_np, pred_labels, zero_division=0); precision = precision_score(y_test_np, pred_labels, zero_division=0)
            recall = recall_score(y_test_np, pred_labels, zero_division=0); accuracy = accuracy_score(y_test_np, pred_labels)
            
            metrics_to_log = {"auc_roc": auc_roc, "auc_pr": auc_pr_val, "logloss": logloss, "brier_score": brier, "f1": f1, "precision": precision, "recall": recall, "accuracy": accuracy}
            for i, (m_name, m_val) in enumerate(sorted(metrics_to_log.items())):
                if i < MAX_METRICS_TO_LOG_PER_RUN: mlflow.log_metric(m_name, m_val)
            
            model_signature = mlflow.models.infer_signature(X_test_np, pd.Series(pred_proba, name=TARGET_COLUMN_NAME))
            if CURRENT_ALGORITHM_TYPE_HPO_OBJ == 'lightgbm': mlflow.lightgbm.log_model(model, "model", signature=model_signature)
            elif CURRENT_ALGORITHM_TYPE_HPO_OBJ == 'xgboost': mlflow.xgboost.log_model(model, "model", signature=model_signature)
            elif CURRENT_ALGORITHM_TYPE_HPO_OBJ == 'catboost': mlflow.catboost.log_model(model, "model", signature=model_signature)
            
            mlflow.set_tag("status", "success")
            loss = float('inf')
            if HPO_OPTIMIZATION_METRIC == 'auc_roc': loss = -auc_roc
            elif HPO_OPTIMIZATION_METRIC == 'auc_pr': loss = -auc_pr_val
            elif HPO_OPTIMIZATION_METRIC == 'log_loss': loss = logloss
            elif HPO_OPTIMIZATION_METRIC == 'brier_score': loss = brier
            else: raise ValueError(f"Unsupported HPO Metric: {HPO_OPTIMIZATION_METRIC}")
            return {'loss': loss, 'status': STATUS_OK, 'run_id': trial_run.info.run_id, 'attachments': metrics_to_log, 'model_instance': model, 'X_test_pdf_for_pdp': X_test_pdf.copy()} # Pass model and X for PDP

        except Exception as e: # ... (Error handling as before) ...
            mlflow.set_tag("status", "failed"); print(f"TRIAL ERROR: {e}"); import traceback; traceback.print_exc();
            return {'loss': float('inf'), 'status': 'fail', 'run_id': trial_run.info.run_id if 'trial_run' in locals() and hasattr(trial_run,'info') else None, 'error_message': str(e)[:250]}
print("Objective function for classification defined.")

# --- Function to plot and log PDP for scikit-learn compatible models ---
def plot_and_log_pdp_sklearn(model_to_plot, X_data_for_pdp_df: pd.DataFrame, feature_name_for_pdp: str, 
                             model_name_pdp: str, target_name_pdp_plot: str, 
                             pdp_grid_points: int, pdp_percentiles_range: tuple):
    if X_data_for_pdp_df.empty or feature_name_for_pdp not in X_data_for_pdp_df.columns:
        print(f"  PDP: Skipping for {feature_name_for_pdp} in {model_name_pdp}, data empty or feature missing."); return
    print(f"  PDP: Generating for '{feature_name_for_pdp}' of model '{model_name_pdp}'...")
    try:
        fig, ax = plt.subplots(figsize=(10, 6))
        # For classifiers, PDP by default shows one line per class if predict_proba is available.
        # We are interested in P(Conversion=1), which is typically class 1.
        # The `from_estimator` method usually handles this by plotting for each class or for the positive class's probability.
        # We might need to ensure that the model's `predict_proba` is used and we plot for the positive class (index 1).
        # If `target_idx=1` or similar is not available, we might plot all or manually get proba for class 1.
        # For most sklearn classifiers, `kind='average'` with `predict_proba` should give P(class=1) if it's binary.
        PartialDependenceDisplay.from_estimator(
            model_to_plot, X_data_for_pdp_df, features=[feature_name_for_pdp],
            # For classifiers and predict_proba, it often plots for each class.
            # We might need to target class 1. If `response_method='predict_proba'`, 
            # then the display might show lines for each class.
            # Let's assume for now it plots for the positive class or we handle it.
            # Often, it's simpler to get P(class=1) and plot that directly if PDP function is tricky.
            # However, from_estimator should be able to handle it.
            # We are interested in the probability of class 1.
            # For binary classification, if model.classes_ = [0, 1], then target_idx=1 for P(Y=1).
            # If from_estimator plots multiple lines for classes, we might need to pick one.
            # For now, let it plot what it defaults to for classifiers (often P(class=1) or both).
            kind='average', n_cols=1, ax=ax,
            n_jobs=-1, grid_resolution=pdp_grid_points, percentiles=pdp_percentiles_range
        )
        ax.set_title(f"PDP: {feature_name_for_pdp} vs. P({target_name_pdp_plot}=1)\nModel: {model_name_pdp}", fontsize=14)
        ax.set_xlabel(feature_name_for_pdp, fontsize=12); ax.set_ylabel(f"Avg. Predicted P({target_name_pdp_plot}=1)", fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7); fig.tight_layout()
        mlflow.log_figure(fig, f"pdp_{model_name_pdp}_{feature_name_for_pdp}.png"); plt.close(fig)
        print(f"    PDP for {feature_name_for_pdp} of {model_name_pdp} logged.")
    except Exception as e_pdp: print(f"    ERROR PDP for {feature_name_for_pdp} of {model_name_pdp}: {e_pdp}")

# --- Function to plot and log Feature Importance ---
def plot_and_log_feature_importance(model_fi, feature_names_fi: List[str], model_name_fi: str, top_n: int = 20):
    print(f"  FeatureImp: Generating for model '{model_name_fi}'...")
    try:
        if hasattr(model_fi, 'feature_importances_'):
            importances = model_fi.feature_importances_
            indices = np.argsort(importances)[::-1][:top_n]
            sorted_feature_names = [feature_names_fi[i] for i in indices]
            
            fig, ax = plt.subplots(figsize=(12, max(6, top_n * 0.3))) # Adjust height
            ax.barh(range(len(indices)), importances[indices][::-1], align='center', color='lightgreen') # Plot sorted
            ax.set_yticks(range(len(indices)))
            ax.set_yticklabels(sorted_feature_names[::-1]) # Match bar order
            ax.set_xlabel("Feature Importance Score", fontsize=12)
            ax.set_ylabel("Feature", fontsize=12)
            ax.set_title(f"Top {top_n} Feature Importances - {model_name_fi}", fontsize=14)
            fig.tight_layout()
            mlflow.log_figure(fig, f"feature_importance_{model_name_fi}.png"); plt.close(fig)
            print(f"    Feature importance plot for {model_name_fi} logged.")
        elif hasattr(model_fi, 'coef_'): # For linear models (meta-learner)
            if model_fi.coef_.ndim > 1: # e.g. LogisticRegression with multi_class='ovr' might have >1 set of coefs
                importances = np.mean(np.abs(model_fi.coef_), axis=0) # Take mean of abs coefs for simplicity
            else:
                importances = np.abs(model_fi.coef_)
            indices = np.argsort(importances)[::-1][:top_n]
            sorted_feature_names = [feature_names_fi[i] for i in indices]

            fig, ax = plt.subplots(figsize=(12, max(6, top_n * 0.3)))
            ax.barh(range(len(indices)), importances[indices][::-1], align='center', color='lightgreen')
            ax.set_yticks(range(len(indices))); ax.set_yticklabels(sorted_feature_names[::-1])
            ax.set_xlabel("Absolute Coefficient Value", fontsize=12); ax.set_ylabel("Feature (Base Model Prediction)", fontsize=12)
            ax.set_title(f"Top {top_n} Feature Importances (Coefficients) - {model_name_fi}", fontsize=14)
            fig.tight_layout()
            mlflow.log_figure(fig, f"feature_importance_coeffs_{model_name_fi}.png"); plt.close(fig)
            print(f"    Feature importance (coefficients) plot for {model_name_fi} logged.")
        else:
            print(f"    Model {model_name_fi} does not have 'feature_importances_' or 'coef_' attribute. Skipping FI plot.")
    except Exception as e_fi: print(f"    ERROR FeatureImp for {model_name_fi}: {e_fi}")


# --- OOF Generation and Final Model Training Function (Classification) ---
def train_final_model_and_generate_oof_classif(
                                     model_type_oof, best_hyperparams_oof,
                                     train_parquet_path_oof, test_parquet_path_oof, 
                                     label_col_name_oof, premium_col_name_oof,
                                     k_folds_oof_val, seed_oof, mlflow_parent_run_name_prefix_oof,
                                     oof_output_dir_ucv_oof, test_preds_output_dir_ucv_oof):
    # ... (Sanitize hyperparams as in objective_function) ...
    sanitized_best_hyperparams = {k: v.item() if isinstance(v, np.generic) else int(v) if k in ['max_depth', 'depth', 'n_estimators', 'num_leaves', 'iterations', 'border_count', 'scale_pos_weight'] and v is not None else (int(v) if isinstance(v,float) and v.is_integer() else v) for k,v in best_hyperparams_oof.items()}
    if 'max_depth' in sanitized_best_hyperparams and sanitized_best_hyperparams['max_depth'] is None: pass
    elif 'max_depth' in sanitized_best_hyperparams: sanitized_best_hyperparams['max_depth'] = int(sanitized_best_hyperparams['max_depth'])
    if 'depth' in sanitized_best_hyperparams: sanitized_best_hyperparams['depth'] = int(sanitized_best_hyperparams['depth'])

    with mlflow.start_run(run_name=f"{mlflow_parent_run_name_prefix_oof}_{model_type_oof}_OOF_FinalModel", nested=False) as oof_parent_run:
        mlflow.log_params(sanitized_best_hyperparams); mlflow.log_param("model_type", model_type_oof); mlflow.log_param("k_folds_for_oof", k_folds_oof_val)
        mlflow.set_tag("seed", seed_oof); mlflow.log_param("train_data_oof", train_parquet_path_oof); mlflow.log_param("test_data_oof", test_parquet_path_oof)
        final_model_run_id = oof_parent_run.info.run_id
        print(f"Starting OOF & Final Model for {model_type_oof}. MLflow Run ID: {final_model_run_id}")

        try:
            X_full_train_pdf, y_full_train_series, train_feature_names = load_processed_parquet_to_sklearn(train_parquet_path_oof, label_col_name_oof)
            X_test_pdf, y_test_series, test_feature_names = load_processed_parquet_to_sklearn(test_parquet_path_oof, label_col_name_oof)
            
            X_full_train_np, y_full_train_np = X_full_train_pdf.values, y_full_train_series.values
            X_test_np, y_test_np = X_test_pdf.values, y_test_series.values if y_test_series is not None else None

            # Monotonicity for OOF/Final model
            oof_monotone_constraints_val = None
            if premium_col_name_oof in train_feature_names:
                premium_idx_oof = train_feature_names.index(premium_col_name_oof)
                oof_monotone_constraints_val = [0] * len(train_feature_names); oof_monotone_constraints_val[premium_idx_oof] = -1
                mlflow.log_param("final_model_mono_constraint_on", f"{premium_col_name_oof}_idx{premium_idx_oof}_is_-1")
            
            oof_pred_probas_np = np.zeros_like(y_full_train_np, dtype=float)
            kf = KFold(n_splits=k_folds_oof_val, shuffle=True, random_state=seed_oof)

            for fold_num, (train_idx, val_idx) in enumerate(kf.split(X_full_train_np, y_full_train_np)):
                print(f"    OOF Fold {fold_num+1}/{k_folds_oof_val} for {model_type_oof}...")
                X_f_train, X_f_val = X_full_train_np[train_idx], X_full_train_np[val_idx]
                y_f_train = y_full_train_np[train_idx]
                
                model_fold_params = sanitized_best_hyperparams.copy()
                if model_type_oof in ['lightgbm', 'xgboost', 'catboost'] and oof_monotone_constraints_val:
                    if model_type_oof == 'xgboost': model_fold_params['monotone_constraints'] = tuple(oof_monotone_constraints_val)
                    else: model_fold_params['monotone_constraints'] = oof_monotone_constraints_val

                model_fold = None # Instantiate model (as in objective_function)
                if model_type_oof == 'lightgbm': model_fold = lgb.LGBMClassifier(**model_fold_params, random_state=seed_oof, n_jobs=-1, verbose=-1)
                elif model_type_oof == 'xgboost': model_fold = xgb.XGBClassifier(**model_fold_params, random_state=seed_oof, use_label_encoder=False, eval_metric='logloss', n_jobs=-1)
                elif model_type_oof == 'catboost': model_fold = cb.CatBoostClassifier(**model_fold_params, random_state=seed_oof, verbose=0, allow_writing_files=False)
                else: raise ValueError(f"Unsupported model type for OOF: {model_type_oof}")
                
                model_fold.fit(X_f_train, y_f_train)
                oof_pred_probas_np[val_idx] = model_fold.predict_proba(X_f_val)[:, 1]
            
            oof_auc_roc_val = roc_auc_score(y_full_train_np, oof_pred_probas_np)
            oof_logloss_val = log_loss(y_full_train_np, oof_pred_probas_np)
            mlflow.log_metric("oof_auc_roc", oof_auc_roc_val); mlflow.log_metric("oof_logloss", oof_logloss_val)
            print(f"    {model_type_oof} OOF AUC_ROC: {oof_auc_roc_val:.4f}, OOF LogLoss: {oof_logloss_val:.4f}")

            # Train final model on ALL training data
            final_model_params = sanitized_best_hyperparams.copy() # Apply monotonicity to final model too
            if model_type_oof in ['lightgbm', 'xgboost', 'catboost'] and oof_monotone_constraints_val:
                if model_type_oof == 'xgboost': final_model_params['monotone_constraints'] = tuple(oof_monotone_constraints_val)
                else: final_model_params['monotone_constraints'] = oof_monotone_constraints_val
            
            final_model = None # Instantiate final model
            if model_type_oof == 'lightgbm': final_model = lgb.LGBMClassifier(**final_model_params, random_state=seed_oof, n_jobs=-1, verbose=-1)
            elif model_type_oof == 'xgboost': final_model = xgb.XGBClassifier(**final_model_params, random_state=seed_oof, use_label_encoder=False, eval_metric='logloss', n_jobs=-1)
            elif model_type_oof == 'catboost': final_model = cb.CatBoostClassifier(**final_model_params, random_state=seed_oof, verbose=0, allow_writing_files=False)
            else: raise ValueError(f"Unsupported model type for final training: {model_type_oof}")

            final_model.fit(X_full_train_np, y_full_train_np)
            final_model_test_pred_probas = final_model.predict_proba(X_test_np)[:, 1]

            # Save OOF (train) and Test predictions (probabilities) as Parquet
            oof_df_to_save = pd.DataFrame({f'oof_pred_proba_{model_type_oof}': oof_pred_probas_np}, index=X_full_train_pdf.index)
            oof_df_to_save[label_col_name_oof] = y_full_train_np
            
            test_preds_df_to_save = pd.DataFrame({f'test_pred_proba_{model_type_oof}': final_model_test_pred_probas}, index=X_test_pdf.index)
            if y_test_np is not None: test_preds_df_to_save[label_col_name_oof] = y_test_np

            os.makedirs(oof_output_dir_ucv_oof, exist_ok=True); os.makedirs(test_preds_output_dir_ucv_oof, exist_ok=True)
            oof_file_path = os.path.join(oof_output_dir_ucv_oof, f"oof_pred_probas_{model_type_oof}.parquet")
            test_preds_file_path = os.path.join(test_preds_output_dir_ucv_oof, f"test_pred_probas_{model_type_oof}.parquet")
            oof_df_to_save.to_parquet(oof_file_path); test_preds_df_to_save.to_parquet(test_preds_file_path) # index=True if index is meaningful
            mlflow.log_artifact(oof_file_path, "oof_predictions_parquet"); mlflow.log_artifact(test_preds_file_path, "test_predictions_parquet")
            mlflow.set_tag(f"oof_pred_probas_path_{model_type_oof}", oof_file_path); mlflow.set_tag(f"test_pred_probas_path_{model_type_oof}", test_preds_file_path)

            if y_test_np is not None:
                final_model_auc = roc_auc_score(y_test_np, final_model_test_pred_probas); mlflow.log_metric("final_model_test_auc_roc", final_model_auc)
                final_model_logloss = log_loss(y_test_np, final_model_test_pred_probas); mlflow.log_metric("final_model_test_logloss", final_model_logloss)
                print(f"    {model_type_oof} Final Model Test AUC_ROC: {final_model_auc:.4f}, LogLoss: {final_model_logloss:.4f}")
            else: final_model_auc, final_model_logloss = None, None
            
            model_signature = mlflow.models.infer_signature(X_test_np, pd.Series(final_model_test_pred_probas, name=TARGET_COLUMN_NAME))
            if model_type_oof == 'lightgbm': mlflow.lightgbm.log_model(final_model, "final_model", signature=model_signature)
            elif model_type_oof == 'xgboost': mlflow.xgboost.log_model(final_model, "final_model", signature=model_signature)
            elif model_type_oof == 'catboost': mlflow.catboost.log_model(final_model, "final_model", signature=model_signature)
            
            # PDP and Feature Importance for the final_model
            plot_and_log_pdp_sklearn(final_model, X_test_pdf, PREMIUM_COLUMN_NAME, f"Final_{model_type_oof}", TARGET_COLUMN_NAME, PDP_N_GRID_POINTS, PDP_PERCENTILES)
            plot_and_log_feature_importance(final_model, test_feature_names, f"Final_{model_type_oof}")

            mlflow.set_tag("status", "success_oof_final")
            return {"status": "success", "model_type": model_type_oof, "final_model_run_id": final_model_run_id,
                    "oof_auc_roc": oof_auc_roc_val, "oof_logloss": oof_logloss_val,
                    "final_model_test_auc_roc": final_model_auc, "final_model_test_logloss": final_model_logloss,
                    "oof_pred_probas_path": oof_file_path, "test_pred_probas_path": test_preds_file_path }
        except Exception as e: # ... Error handling ...
             print(f"ERROR OOF/Final for {model_type_oof}: {e}"); import traceback; traceback.print_exc()
             mlflow.set_tag("status", "failed_oof_final"); mlflow.log_param("error_oof_final", str(e)[:250])
             return {"status": "failed", "model_type": model_type_oof, "error_message": str(e)}


# --- Function to Optimize Ensemble Weights for AUC ---
def optimize_ensemble_weights_auc(oof_pred_probas_df: pd.DataFrame, y_true_oof: np.ndarray, model_names: List[str]):
    """Finds optimal weights for ensemble to maximize AUC on OOF predictions."""
    print(f"  Optimizing ensemble weights for models: {model_names} to maximize AUC...")
    
    oof_preds_array = oof_pred_probas_df[[f"oof_pred_proba_{m}" for m in model_names]].values
    
    def auc_objective(weights):
        if not (0.999 <= np.sum(weights) <= 1.001): # Allow for small float precision issues
            return 2.0 # Penalize if weights don't sum to 1 (since we want to maximize AUC, loss is -AUC)
        
        weighted_oof_preds = np.sum(oof_preds_array * weights, axis=1)
        # Clip probabilities to avoid issues with log_loss if used, also good for AUC
        weighted_oof_preds = np.clip(weighted_oof_preds, 1e-15, 1 - 1e-15)
        auc = roc_auc_score(y_true_oof, weighted_oof_preds)
        return -auc # We minimize -AUC to maximize AUC

    num_models = len(model_names)
    initial_weights = np.array([1.0 / num_models] * num_models)
    bounds = [(0, 1)] * num_models # Weights between 0 and 1
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}) # Weights sum to 1

    result = minimize(auc_objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)

    if result.success:
        optimized_weights = result.x
        optimized_auc = -result.fun
        print(f"    Optimized weights: {dict(zip(model_names, optimized_weights))}, Optimized OOF AUC: {optimized_auc:.4f}")
        return dict(zip(model_names, optimized_weights)), optimized_auc
    else:
        print(f"    WARNING: Ensemble weight optimization failed or did not converge. Message: {result.message}. Using equal weights.")
        equal_weights = dict(zip(model_names, initial_weights))
         # Recalculate AUC with equal weights for reporting
        weighted_oof_preds_equal = np.sum(oof_preds_array * initial_weights, axis=1)
        weighted_oof_preds_equal = np.clip(weighted_oof_preds_equal, 1e-15, 1 - 1e-15)
        equal_auc = roc_auc_score(y_true_oof, weighted_oof_preds_equal)
        return equal_weights, equal_auc


# --- Pyfunc Model for Weighted Ensemble ---
class WeightedEnsembleClassifierPyfunc(mlflow.pyfunc.PythonModel):
    def __init__(self, base_model_details_for_pyfunc: Dict[str, str], # {model_type: model_run_id}
                       weights_for_pyfunc: Dict[str, float],
                       feature_names_for_pyfunc: List[str], # For signature and internal use
                       premium_col_name_for_pyfunc: str, # For PDP
                       target_col_name_for_pyfunc: str # For signature
                       ):
        self.base_model_details = base_model_details_for_pyfunc
        self.weights = weights_for_pyfunc
        self.feature_names = feature_names_for_pyfunc
        self.premium_col_name_pyfunc = premium_col_name_for_pyfunc
        self.target_col_name_pyfunc = target_col_name_for_pyfunc
        self.loaded_base_models = {} # To store loaded models in context

    def load_context(self, context):
        print("WeightedEnsemblePyfunc: Loading base models...")
        for model_type, model_run_id in self.base_model_details.items():
            model_uri = f"runs:/{model_run_id}/final_model" # Assuming "final_model" is the artifact path
            print(f"  Loading base model: {model_type} from {model_uri}")
            try:
                if model_type == 'lightgbm': self.loaded_base_models[model_type] = mlflow.lightgbm.load_model(model_uri)
                elif model_type == 'xgboost': self.loaded_base_models[model_type] = mlflow.xgboost.load_model(model_uri)
                elif model_type == 'catboost': self.loaded_base_models[model_type] = mlflow.catboost.load_model(model_uri)
                else: self.loaded_base_models[model_type] = mlflow.sklearn.load_model(model_uri)
                print(f"    {model_type} loaded successfully.")
            except Exception as e:
                print(f"    ERROR loading base model {model_type} from {model_uri}: {e}")
                # Decide how to handle: raise error or allow ensemble to work with fewer models?
                # For now, let it raise, as weights are specific to the full set.
                raise
        if not self.loaded_base_models or len(self.loaded_base_models) != len(self.weights):
            raise ValueError("Mismatch between expected base models for weights and loaded models.")
        print("WeightedEnsemblePyfunc: All base models loaded.")


    def predict(self, context, model_input_pdf: pd.DataFrame):
        print(f"WeightedEnsemblePyfunc: Predicting on input shape {model_input_pdf.shape}")
        # Ensure model_input_pdf has the correct features in the correct order if models expect NumPy
        # If base models were trained on X_np which came from X_pdf.values, then the order is preserved.
        # Our base models were trained on X_np derived from X_pdf.values where X_pdf had feature_names_ordered.
        # For safety, reorder input if self.feature_names is reliable
        
        X_input_np = None
        if self.feature_names and all(col in model_input_pdf.columns for col in self.feature_names):
            X_input_np = model_input_pdf[self.feature_names].values
        else: # Fallback if feature_names not perfectly set, or input is already just features
            X_input_np = model_input_pdf.values
            if X_input_np.shape[1] != len(self.feature_names) and self.feature_names:
                 print(f"  Warning: Input columns for ensemble predict ({model_input_pdf.columns.tolist()}) do not match expected feature_names ({self.feature_names}). Using raw values.")


        final_predictions = np.zeros(len(X_input_np))
        total_weight_applied = 0.0

        for model_type, model_instance in self.loaded_base_models.items():
            if model_type in self.weights:
                weight = self.weights[model_type]
                base_pred_proba = model_instance.predict_proba(X_input_np)[:, 1] # Probability of class 1
                final_predictions += weight * base_pred_proba
                total_weight_applied += weight
            else:
                print(f"  Warning: No weight found for loaded base model {model_type}. Skipping its contribution.")
        
        # Normalize if total_weight_applied is slightly off 1 due to float precision or filtering
        if total_weight_applied > 1e-6 and not (0.999 <= total_weight_applied <= 1.001):
            print(f"  Normalizing final_predictions as total_weight_applied is {total_weight_applied}")
            final_predictions /= total_weight_applied
        
        final_predictions = np.clip(final_predictions, 0.0, 1.0) # Ensure valid probabilities
        print(f"WeightedEnsemblePyfunc: Prediction complete.")
        return pd.Series(final_predictions)


# --- Function to plot ensemble weights ---
def plot_and_log_ensemble_weights(weights_dict: Dict[str, float], ensemble_name: str):
    print(f"  Plotting weights for {ensemble_name}...")
    try:
        names = list(weights_dict.keys())
        values = list(weights_dict.values())
        fig, ax = plt.subplots(figsize=(max(8, len(names) * 1.5), 5))
        ax.bar(names, values, color='teal')
        ax.set_ylabel("Optimized Weight", fontsize=12)
        ax.set_xlabel("Base Model Type", fontsize=12)
        ax.set_title(f"Optimized Ensemble Weights - {ensemble_name}", fontsize=14)
        plt.xticks(rotation=45, ha="right")
        fig.tight_layout()
        mlflow.log_figure(fig, f"ensemble_weights_{ensemble_name.replace(' ', '_')}.png")
        plt.close(fig)
        print(f"    Ensemble weights plot for {ensemble_name} logged.")
    except Exception as e_plot_weights:
        print(f"    ERROR plotting ensemble weights for {ensemble_name}: {e_plot_weights}")

print("--- All Utility Functions & Core Logic (HPO, OOF, Ensemble) Defined ---")
print("-" * 50)

# <-------------------- CELL 4: MAIN ORCHESTRATION LOGIC (HPO, OOF, Ensembling) -------------------->
print("\nCell 4: Main Orchestration Logic - Executing...")

# --- 0. Setup MLflow Experiment ---
global main_training_mlflow_experiment_id
main_training_mlflow_experiment_id = None
try:
    main_training_mlflow_experiment_id = get_or_create_experiment(MLFLOW_EXPERIMENT_PATH, spark)
    if main_training_mlflow_experiment_id:
        mlflow.set_experiment(experiment_id=main_training_mlflow_experiment_id)
        print(f"MLflow experiment '{MLFLOW_EXPERIMENT_PATH}' for Training is set with ID: {main_training_mlflow_experiment_id}")
    else: raise Exception("Main Training MLflow experiment could not be set. Halting.")
except Exception as e: print(f"CRITICAL: Could not initialize main MLflow experiment. Error: {e}") # Halt

if main_training_mlflow_experiment_id:
    # These globals are used by the objective function
    global HPO_PARENT_RUN_ID_HPO_OBJ, CURRENT_ALGORITHM_TYPE_HPO_OBJ, FEATURE_NAMES_FOR_HPO_OBJ

    # --- 1. Individual HPO for each Base Algorithm (Sequential) ---
    print("\n--- Phase 1: Individual Hyperparameter Optimization for Base Models (Sequential) ---")
    best_hpo_configs_per_algorithm = {} 
    best_hpo_models_for_pdp = {} # Store {'algo_type': model_instance} for PDP
    X_test_pdf_for_pdp, _, test_feature_names_for_pdp = load_processed_parquet_to_sklearn(SHARED_PROCESSED_TEST_PATH, TARGET_COLUMN_NAME)


    for algo_type_hpo in BASE_ALGORITHMS_TO_TRAIN:
        print(f"\nStarting HPO Campaign for Algorithm: {algo_type_hpo}...")
        if algo_type_hpo not in ALGORITHM_CLASSIFIER_SEARCH_SPACES:
            print(f"  Warning: Search space for {algo_type_hpo} not defined. Skipping HPO."); continue

        with mlflow.start_run(run_name=f"HPO_Campaign_{algo_type_hpo}", nested=False) as hpo_campaign_run:
            HPO_PARENT_RUN_ID_HPO_OBJ = hpo_campaign_run.info.run_id
            CURRENT_ALGORITHM_TYPE_HPO_OBJ = algo_type_hpo
            
            mlflow.log_params({ # Log main HPO config
                "hpo_algorithm_target": algo_type_hpo, "num_hpo_trials_config": NUM_HPO_TRIALS_PER_ALGO,
                "primary_metric_config": HPO_OPTIMIZATION_METRIC, "global_seed_config": GLOBAL_SEED
            })
            try: mlflow.log_dict({k:str(v) for k,v in ALGORITHM_CLASSIFIER_SEARCH_SPACES[algo_type_hpo]['model_params'].items()}, f"search_space_{algo_type_hpo}.json")
            except Exception as log_e: print(f"  Warning: Could not log search space: {log_e}")

            hpo_trials_db = Trials()
            try:
                current_search_space = ALGORITHM_CLASSIFIER_SEARCH_SPACES[algo_type_hpo]['model_params']
                print(f"  Running fmin for {algo_type_hpo} with {NUM_HPO_TRIALS_PER_ALGO} trials...")
                best_indices = fmin(fn=objective_function_classification, space=current_search_space, algo=tpe.suggest,
                                    max_evals=NUM_HPO_TRIALS_PER_ALGO, trials=hpo_trials_db, rstate=np.random.default_rng(GLOBAL_SEED))
                
                best_params = space_eval(current_search_space, best_indices)
                best_trial = hpo_trials_db.best_trial
                if best_trial and best_trial['result']['status'] == STATUS_OK:
                    best_trial_run_id = best_trial['result'].get('run_id')
                    best_loss = best_trial['result']['loss']
                    attachments = best_trial['result'].get('attachments', {})
                    print(f"    Best HPO trial for {algo_type_hpo}: Loss={best_loss:.4f}, Params={best_params}, MLflow Trial Run ID={best_trial_run_id}")
                    mlflow.log_params({f"best_hpo_{k}": v for k,v in best_params.items()})
                    mlflow.log_metric("best_hpo_loss_campaign", best_loss)
                    if best_trial_run_id: mlflow.set_tag("best_hpo_trial_run_id", best_trial_run_id)
                    for att_k, att_v in attachments.items():
                        if isinstance(att_v, (int, float)) and att_k != "model_type": mlflow.log_metric(f"best_hpo_trial_{att_k}", att_v)
                    
                    best_hpo_configs_per_algorithm[algo_type_hpo] = {"best_params": best_params, "best_trial_run_id": best_trial_run_id, "hpo_campaign_run_id": HPO_PARENT_RUN_ID_HPO_OBJ, "attachments": attachments, "best_trial_model_instance": best_trial['result'].get('model_instance')}
                    mlflow.set_tag("status_hpo_campaign", "success")
                    
                    # PDP for the best model from this HPO campaign
                    best_model_instance_for_pdp = best_trial['result'].get('model_instance')
                    X_test_pdf_for_pdp_local = best_trial['result'].get('X_test_pdf_for_pdp') # Get X_test_pdf from objective
                    if best_model_instance_for_pdp and X_test_pdf_for_pdp_local is not None and PREMIUM_COLUMN_NAME in X_test_pdf_for_pdp_local.columns:
                        plot_and_log_pdp_sklearn(best_model_instance_for_pdp, X_test_pdf_for_pdp_local, PREMIUM_COLUMN_NAME,
                                                 f"BestHPO_{algo_type_hpo}", TARGET_COLUMN_NAME, PDP_N_GRID_POINTS, PDP_PERCENTILES)
                        plot_and_log_feature_importance(best_model_instance_for_pdp, list(X_test_pdf_for_pdp_local.columns), f"BestHPO_{algo_type_hpo}")
                    else: print(f"    Could not generate PDP/FI for BestHPO_{algo_type_hpo}, model instance or test data for PDP missing from trial result attachments.")

                else: print(f"    HPO for {algo_type_hpo} no successful best trial."); mlflow.set_tag("status_hpo_campaign", "no_successful_best_trial")
            except Exception as e_fmin: # ... error handling for fmin ...
                print(f"  ERROR HPO fmin for {algo_type_hpo}: {e_fmin}"); import traceback; traceback.print_exc(); mlflow.set_tag("status_hpo_campaign", "fmin_error"); mlflow.log_param("error_fmin", str(e_fmin)[:250])
    print("--- Individual HPO Phase Completed ---")

    # --- 2. OOF Generation & Final Base Model Training ---
    print("\n--- Phase 2: OOF Prediction Generation & Final Base Model Training ---")
    final_base_model_outputs = {}
    for algo_type_oof, hpo_data in best_hpo_configs_per_algorithm.items():
        if hpo_data and hpo_data.get("best_params"):
            print(f"\nGenerating OOF & Final Model for: {algo_type_oof}...")
            oof_result = train_final_model_and_generate_oof_classif(
                model_type_oof=algo_type_oof, best_hyperparams_oof=hpo_data['best_params'],
                train_parquet_path_oof=SHARED_PROCESSED_TRAIN_PATH, test_parquet_path_oof=SHARED_PROCESSED_TEST_PATH,
                label_col_name_oof=TARGET_COLUMN_NAME, premium_col_name_oof=PREMIUM_COLUMN_NAME,
                k_folds_oof_val=K_FOLDS_OOF, seed_oof=GLOBAL_SEED, mlflow_parent_run_name_prefix_oof="MVP",
                oof_output_dir_ucv_oof=OOF_PREDS_DIR_UCV, test_preds_output_dir_ucv_oof=TEST_PREDS_DIR_UCV
            )
            if oof_result['status'] == 'success': final_base_model_outputs[algo_type_oof] = oof_result
            else: print(f"  Failed OOF/final model for {algo_type_oof}: {oof_result.get('error_message')}")
        else: print(f"  Skipping OOF for {algo_type_oof}, no successful HPO result.")
    print("--- OOF Generation & Final Base Model Training Phase Completed ---")

    # --- 3. Weighted Ensemble Creation ---
    print("\n--- Phase 3: Weighted Ensemble Creation ---")
    if not final_base_model_outputs: print("No base models for ensembling. Skipping.")
    else:
        y_true_test_for_ensemble_eval_np = None # Load true test labels for evaluating ensembles
        try:
            _, y_true_test_for_ensemble_eval_series, _ = load_processed_parquet_to_sklearn(SHARED_PROCESSED_TEST_PATH, TARGET_COLUMN_NAME)
            if y_true_test_for_ensemble_eval_series is not None: y_true_test_for_ensemble_eval_np = y_true_test_for_ensemble_eval_series.values
            else: print("    Warning: Could not load true test labels for ensemble evaluation.")
        except Exception as e_label: print(f"    Error loading true test labels for ensemble: {e_label}")

        # Define ensemble combinations (all pairs, and all three)
        algo_names_for_ensemble = [algo for algo, res in final_base_model_outputs.items() if res['status'] == 'success']
        dynamic_ensemble_combinations = []
        if len(algo_names_for_ensemble) >= 2:
            for i in range(2, len(algo_names_for_ensemble) + 1):
                for combo in itertools.combinations(algo_names_for_ensemble, i):
                    dynamic_ensemble_combinations.append(combo)
        
        print(f"  Will attempt to create weighted ensembles for combinations: {dynamic_ensemble_combinations}")

        for combo_idx, model_combo_tuple in enumerate(dynamic_ensemble_combinations):
            current_combo_model_types = list(model_combo_tuple)
            combo_name = "_".join(current_combo_model_types)
            print(f"\n  Creating Weighted Ensemble for combination: {combo_name} ({combo_idx+1}/{len(dynamic_ensemble_combinations)})")

            # Gather OOF predictions and test predictions for current combination
            combo_oof_dfs = []
            combo_test_dfs = []
            combo_oof_metrics_for_weighting = []
            valid_models_in_combo_for_meta_features = []

            # Load all OOF predictions once to get y_meta_train
            # (create_ensemble_meta_features_from_parquet can be adapted or its logic used here)
            # For simplicity, let's assume OOF predictions are DFs with one pred column each, and a label col
            
            temp_X_meta_train_pdf, temp_y_meta_train_np, temp_X_meta_test_pdf, _ = create_ensemble_meta_features_from_parquet(
                base_model_types_ens=current_combo_model_types, # Only models in current combo
                oof_pred_dir_ucv_ens=OOF_PREDS_DIR_UCV,
                test_pred_dir_ucv_ens=TEST_PREDS_DIR_UCV,
                label_col_name_in_oof_ens=TARGET_COLUMN_NAME,
                test_true_labels_series_ens=pd.Series(y_true_test_for_ensemble_eval_np, name=TARGET_COLUMN_NAME) if y_true_test_for_ensemble_eval_np is not None else None
            )

            if temp_X_meta_train_pdf is None or temp_y_meta_train_np is None:
                print(f"    Could not create meta-features for combo {combo_name}. Skipping.")
                continue
            
            # Ensure correct columns are selected for weight optimization based on combo
            oof_pred_cols_for_combo = [f"oof_pred_proba_{m}" for m in current_combo_model_types if f"oof_pred_proba_{m}" in temp_X_meta_train_pdf.columns]
            if len(oof_pred_cols_for_combo) != len(current_combo_model_types):
                print(f"    Warning: Mismatch in OOF pred columns for combo {combo_name}. Expected {len(current_combo_model_types)}, found {len(oof_pred_cols_for_combo)}. Skipping.")
                continue
            
            current_oof_pred_probas_df_for_optim = temp_X_meta_train_pdf[oof_pred_cols_for_combo]


            # Get OOF metrics for only the models in the current combination
            for algo_type in current_combo_model_types:
                if algo_type in final_base_model_outputs and final_base_model_outputs[algo_type]['status'] == 'success':
                     combo_oof_metrics_for_weighting.append({
                         'model_type': algo_type,
                         'oof_auc_roc': final_base_model_outputs[algo_type].get('oof_auc_roc'),
                         'oof_logloss': final_base_model_outputs[algo_type].get('oof_logloss')
                         # Add primary metric for weighting here based on HPO_OPTIMIZATION_METRIC
                     })
                else: # Should not happen if already filtered by ensemble_base_model_types_list
                    print(f"    Warning: Model {algo_type} for combo {combo_name} missing from final_base_model_outputs or was not successful.")


            if not combo_oof_metrics_for_weighting or len(combo_oof_metrics_for_weighting) != len(current_combo_model_types) :
                print(f"    Not enough valid OOF metrics for weighting combo {combo_name}. Skipping.")
                continue

            optimized_weights_dict, optimized_oof_auc = optimize_ensemble_weights_auc(
                oof_pred_probas_df=current_oof_pred_probas_df_for_optim, # DF with OOF pred probas for current combo models
                y_true_oof=temp_y_meta_train_np, # True labels for OOF set
                model_names=current_combo_model_types # Names of models in current combo
            )

            with mlflow.start_run(run_name=f"Ensemble_Weighted_{combo_name}", nested=False) as ens_run:
                mlflow.set_tag("ensemble_type", "weighted_average"); mlflow.set_tag("base_models_in_ensemble", combo_name)
                mlflow.log_params({f"weight_{m}": w for m, w in optimized_weights_dict.items()})
                mlflow.log_metric("optimized_ensemble_oof_auc", optimized_oof_auc)
                
                # Apply weights to test predictions for this combo
                test_pred_cols_for_combo = [f"test_pred_proba_{m}" for m in current_combo_model_types if f"test_pred_proba_{m}" in temp_X_meta_test_pdf.columns]
                if len(test_pred_cols_for_combo) != len(current_combo_model_types):
                    print(f"    Warning: Mismatch in test pred columns for weighted ensemble {combo_name}. Skipping evaluation.")
                    mlflow.set_tag("status", "skipped_test_pred_mismatch")
                    continue
                
                current_test_pred_probas_df_for_eval = temp_X_meta_test_pdf[test_pred_cols_for_combo]
                
                # Ensure weights are applied in the correct order
                weights_array = np.array([optimized_weights_dict[model_name] for model_name in current_combo_model_types])
                ensemble_test_pred_probas = np.sum(current_test_pred_probas_df_for_eval.values * weights_array, axis=1)
                ensemble_test_pred_probas = np.clip(ensemble_test_pred_probas, 0.0, 1.0)

                if y_true_test_for_ensemble_eval_np is not None:
                    ens_auc = roc_auc_score(y_true_test_for_ensemble_eval_np, ensemble_test_pred_probas)
                    ens_logloss = log_loss(y_true_test_for_ensemble_eval_np, ensemble_test_pred_probas)
                    mlflow.log_metrics({"ensemble_test_auc_roc": ens_auc, "ensemble_test_logloss": ens_logloss})
                    print(f"    Weighted Ens ({combo_name}) Test AUC: {ens_auc:.4f}, LogLoss: {ens_logloss:.4f}")
                else:
                    print(f"    Weighted Ens ({combo_name}) predictions generated, no true test labels for metrics.")

                # Package this specific weighted ensemble as PyFunc
                base_model_uris_for_pyfunc = {
                    mt: f"runs:/{final_base_model_outputs[mt]['final_model_run_id']}/final_model" 
                    for mt in current_combo_model_types if mt in final_base_model_outputs
                }
                
                # Get feature names from one of the base model's X_test_pdf (they all see same features)
                # This assumes X_test_pdf was loaded earlier for PDP for example.
                # Let's load one test pdf to get feature names.
                _, _, current_feature_names = load_processed_parquet_to_sklearn(SHARED_PROCESSED_TEST_PATH, TARGET_COLUMN_NAME)


                pyfunc_ensemble = WeightedEnsembleClassifierPyfunc(
                    base_model_details_for_pyfunc=base_model_uris_for_pyfunc,
                    weights_for_pyfunc=optimized_weights_dict,
                    feature_names_for_pyfunc=current_feature_names, # Pass feature names
                    premium_col_name_for_pyfunc=PREMIUM_COLUMN_NAME,
                    target_col_name_for_pyfunc=TARGET_COLUMN_NAME
                )
                # Define conda_env for pyfunc
                ens_conda_env = { 'channels': ['conda-forge', 'defaults'], 'dependencies': [f'python={pd.__version__.split(".")[0]}.{pd.__version__.split(".")[1]}', 'pip',
                    {'pip': [f'mlflow>={mlflow.__version__}', f'pandas>={pd.__version__}', f'numpy>={np.__version__}', f'scikit-learn>={sklearn.__version__}', f'lightgbm>={lgb.__version__}', f'xgboost>={xgb.__version__}', f'catboost>={cb.__version__}']}],
                    'name': f'weighted_ens_{combo_name}_env'
                }
                # Need preprocessed test data (features only) for signature and PDP
                X_test_pdf_for_ens_pdp, _, _ = load_processed_parquet_to_sklearn(SHARED_PROCESSED_TEST_PATH, TARGET_COLUMN_NAME)

                # Infer signature using the pyfunc model itself on a sample
                sample_input_for_sig = X_test_pdf_for_ens_pdp.head()
                try:
                    sample_output_for_sig = pyfunc_ensemble.predict(None, sample_input_for_sig)
                    ens_signature = mlflow.models.infer_signature(sample_input_for_sig, pd.Series(sample_output_for_sig, name=TARGET_COLUMN_NAME))
                except Exception as sig_e:
                    print(f"    Warning: Could not infer signature for ensemble {combo_name}: {sig_e}")
                    ens_signature = None

                mlflow.pyfunc.log_model(
                    artifact_path=f"weighted_ensemble_{combo_name}",
                    python_model=pyfunc_ensemble,
                    conda_env=ens_conda_env,
                    signature=ens_signature,
                    input_example=sample_input_for_sig if ens_signature else None
                )
                mlflow.set_tag("status", "success_ensemble_packaged")
                print(f"    Weighted Ensemble Pyfunc model for {combo_name} logged.")

                # PDP for this weighted ensemble pyfunc model
                # This requires loading the pyfunc model and then calling plot_and_log_pdp_sklearn
                # For simplicity in this script, we can generate PDP by directly using the predict logic
                # with varied premium on X_test_pdf_for_ens_pdp
                # Or, more robustly, load the logged pyfunc model and then plot PDP.
                # Let's defer PDP for pyfunc ensembles for now to keep this main script less complex
                # but user knows they need to load the pyfunc and then call PDP function with it.
                # We can log the weights plot.
                plot_and_log_ensemble_weights(optimized_weights_dict, f"WeightedEns_{combo_name}")


        else: # No valid base models after OOF
            print("  Skipping ensemble creation as no base models passed the OOF stage successfully.")

else: # MLflow experiment not set
    print("Halting script because main MLflow experiment for Training could not be set.")

print("\n--- FULL TRAINING & ENSEMBLING ORCHESTRATION COMPLETED ---")